In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
#model.add(kr.layers.Dense(units=100, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
#getting the file info ready to train the model
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
#formatting the data to see if we can acheieve better results
inputs[inputs>0] = 5.5
inputs[inputs<1] = 0 

In [7]:
#label the output layer
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
#just for looking
#print(inputs[0])
#print (inputs[0].size)

In [8]:
#view of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


In [9]:
##start training the neural network
model.fit(inputs, outputs, epochs=5, batch_size=150)

Epoch 1/5
60000/60000 [==============================] - 17s 290us/step - loss: 0.2434 - accuracy: 0.9317
Epoch 2/5
60000/60000 [==============================] - 14s 232us/step - loss: 0.0918 - accuracy: 0.9715
Epoch 3/5
60000/60000 [==============================] - 14s 226us/step - loss: 0.0665 - accuracy: 0.9785
Epoch 4/5
60000/60000 [==============================] - 13s 221us/step - loss: 0.0493 - accuracy: 0.9839
Epoch 5/5
60000/60000 [==============================] - 13s 219us/step - loss: 0.0416 - accuracy: 0.9864


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
#checking the file
#type(testImages)

In [12]:
#checking the data
#testImages[0:4]

In [13]:
#prepare the test images for prediction
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
testInputs[testInputs>0] = 1
testInputs[testInputs<1] = 0 
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how many it guessed correctly
(predictions == encoder.inverse_transform(testOutputs)).sum()

9751

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[7.97412824e-03, 1.68111324e-02, 9.36195254e-03, ...,
        8.69793117e-01, 6.21759472e-03, 6.03362955e-02],
       [3.08564282e-03, 1.14536777e-01, 8.40862751e-01, ...,
        1.21319899e-02, 6.38479600e-03, 7.63053773e-04],
       [1.03505915e-02, 7.62610495e-01, 2.60422900e-02, ...,
        3.32697369e-02, 6.47545010e-02, 1.24157192e-02],
       ...,
       [5.74759766e-03, 1.18651045e-02, 1.35918688e-02, ...,
        3.98940966e-02, 1.29854986e-02, 3.12344432e-02],
       [9.74255032e-04, 8.98444327e-04, 9.73894144e-04, ...,
        4.44674602e-04, 4.51072305e-02, 1.03313988e-03],
       [8.60314630e-03, 7.34685967e-03, 1.24263717e-02, ...,
        2.69935722e-03, 2.64261942e-02, 3.11613688e-03]], dtype=float32)

In [16]:
##Model complete

In [17]:
##Re-train with an extra layer

In [18]:
type(model)

keras.engine.sequential.Sequential

In [19]:
##loop through old model
#and add all the trained layers to a new model and make them un-trainable
fineTune = kr.models.Sequential()
for layer in model.layers:
    fineTune.add(layer)
for layer in fineTune.layers:
    layer.trainable = False

In [20]:
#add an extra output layer
#with sigmoid activation
#refine the guess once more
fineTune.add(kr.layers.Dense(units=10, activation='sigmoid'))

In [21]:
#compile and re-train
fineTune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fineTune.fit(inputs, outputs, epochs=5, batch_size=50)

Epoch 1/5
60000/60000 [==============================] - 4s 65us/step - loss: 0.4077 - accuracy: 0.8960
Epoch 2/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.1550 - accuracy: 0.9906
Epoch 3/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.0808 - accuracy: 0.9981
Epoch 4/5
60000/60000 [==============================] - 4s 60us/step - loss: 0.0475 - accuracy: 0.9982
Epoch 5/5
60000/60000 [==============================] - 4s 60us/step - loss: 0.0299 - accuracy: 0.9982


In [22]:
# Have the network predict the classes of the test inputs.
predictions = fineTune.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how many it guesses correctly
(predictions == encoder.inverse_transform(testOutputs)).sum()

9753

In [23]:
#Results
#save new model
fineTune.save('savedModel/freshlyTrained.h5')
#test in client